<img src="Figuras - Desafio 3/logo.png" style="width:1450px;height:250px"/>

# Ilum Escola de Ciência
### Curso: Termodinâmica Avançada
### Prof: Amauri Jardim de Paula
### Alunos: Gabriel Pereira e Isabela Beneti
### Desafio 3

#
## Descrição do desafio
O terceiro desafio para solução do problema principal da disciplina envolve:

#### **A combustão dos compostos escolhidos**

## Problemas

#### Para solucionar o desafio, alguns aspectos importantes devem ser considerados.

##### A. Os potenciais termodinâmicos (energia interna, entalpia, energia livre de Gibbs) envolvidos na reação de combustão.
##### B. A entropia envolvida na reação de combustão.
##### C.A equação de estado a ser usada para o sistema.

#
## Resoluções

### Importações

In [1]:
# Rode essa célula antes de rodar as seguintes, pois elas se utilizam das seguintes bibliotecas:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from funcoes_desafio_3 import *
import imageio
import warnings

### Esquema do Curso e Desafios

<center>
  <img src='./Figuras - Desafio 3/Esquema do Curso.png' style="width:900px;height:500px"/>
    
    Figura 1: esquema do curso e desafios
</center>

### Recapitulação

Antes de começarmos o terceiro desafio, é importante retomar o que foi feito no desafio anterior e o que obtemos como resultado dele:

No desafio anterior, nós separamos a mistura inicial- definida no desafio 1 como uma mistura de água com etanol a 40% v/v- por meio de destilação fracionada. Como resultado, obtivemos dois líquidos: uma mistura de etanol enriquecido (em fração molar 0.773) de massa 384.73 kg e 615.27 kg de água pura. 

Após disso, nós evaporamos o etanol enriquecido a 81.54°C, o que gerou **384.73 kg de etanol enriquecido gasoso**. Por fim, realizamos eletrólise da água, através da qual extraímos **47.85 kg de hidrogênio gasoso**.

É importante pontuar, também, que o processo de evaporação do útimo desafio foi realizado à pressão constante de 1 atm, num recipiente semelhante a um **balão**. Para esse próximo desafio, transferiremos o conteúdo de ambos os balões para um reator de **volume constante**.

### Problema A

#### Objetivo: 
 Calcular os potenciais termodinâmicos (energia interna, entalpia, energia livre de Gibbs) envolvidos na reação de combustão.

#### O que temos: 
- 


#### O que iremos fazer:
- 

### Entalpia de Combustão:

A reação de combustão completa do etanol é dada por:



##
### Problema B

#### Objetivo
Determinar a entropia envolvida na reação de combustão.

#### O que temos:
- 

#### O que iremos fazer
- 

### Problema C

#### Objetivo

Determinar a equação de estado a ser usada para o sistema.


#### O que temos:
- 

#### O que iremos fazer
- 

# Acesse o repositório do projeto!

Para melhor compreensão de cade etapa e de cada escolha do nosso projeto, acesse, clicando aqui ou no link abaixo, nosso [Repositório do Github](https://github.com/benetao/Termodinamica_Avancada), onde estão armazenados todos os notebooks e informações referentes aos desafios:

https://github.com/benetao/Termodinamica_Avancada

#
## Referências
- Livro An Introduction to Thermal Physics, Daniel V. Schroeder, Editora Addison-Wesley.
- Livro Physical Chemistry: A Molecular Approach, Donald A. McQuarrie, Editora Univesity Science Books.
- Livro Richard Feynman, Lições de Física Vol 1, Editora Bookman.
        Capítulo de Teoria Cinética dos Gases